In [1]:
#import engie - responsible for creating connections
from sqlalchemy import create_engine, Column, Integer, String, or_, func, ForeignKey, select
# declarative base used to create models
from sqlalchemy.orm import declarative_base, sessionmaker, relationship, mapped_column, Mapped, DeclarativeBase
from typing import Optional
# common syntax for the connection url
# "<dialect>+<driver>://<usermname>:<password>@<host>:<port>/database"
# postfresql
# "postgresql+psycopg2://<username>:<password>@<hostname>:<port>/database"
# mysql
# "mysql://<username>:<password>@<hostname>:<port>/database"
#sqllite
# /// - relative path & //// - absolute path
db_url = "sqlite:///CivilPM.db"
engine = create_engine(db_url)
# Session maker
Session = sessionmaker(bind=engine)
# initiate session to create actions
session = Session()
# Base = DeclarativeBase()

In [2]:
class Base(DeclarativeBase):
    pass

class User(Base): 
    __tablename__ = "users"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    email: Mapped[str]
    password: Mapped[Optional[str]]
    boq_list_id: Mapped[list["BOQ"]] = relationship()    

class BOQ(Base):
    __tablename__ = "boqs"
    id: Mapped[int] = mapped_column(primary_key=True)
    client_id: Mapped["Client"] = relationship()   
    category: Mapped["Category"]  = relationship()  
    stages: Mapped[list[str]] = relationship()
    resources: Mapped[list[str]] = relationship()
    quantity: Mapped[Optional[str]]
    unit: Mapped[Optional[str]]
    rate: Mapped[Optional[float]]
    amount: Mapped[Optional[float]]
    user_id: Mapped[Optional[int]] = mapped_column(ForeignKey("users.id"))

class Client(Base):
    __tablename__ = "client_data"
    id: Mapped[int] = mapped_column(primary_key=True)
    client_name: Mapped[Optional[str]]
    total_sqft: Mapped[Optional[float]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))

class Category(Base):
    __tablename__ = "categories"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    description: Mapped[Optional[str]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))
  
class Stage(Base):
    __tablename__ = "stages"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    description: Mapped[Optional[str]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))

class Resource(Base):
    __tablename__ = "resources"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    category: Mapped[str]
    price: Mapped[float]
    unit: Mapped[str]    
    description: Mapped[Optional[str]]
    boq_id: Mapped[Optional[int]] = mapped_column(ForeignKey("boqs.id"))


# this will create DB and tables
Base.metadata.create_all(engine)


In [3]:
# Create entries in table
user = User(name="Paneer", age=44)
user_2 = User(name="Radha",age=42)
user_3 = User(name="Neha", age=17)
task_1 = Task(title="Test task1", description="test description")
# add single object to session
session.add(user)
# add multiple objects to sesson
session.add_all([user_2,user_3, task_1])
user.tasks.extend([task_1])

#commit changes
session.commit()


user.tasks =[<__main__.Task object at 0x00000196A3BF7EC0>]


In [7]:
print(f"{user.tasks[0].title}")

Test task1


In [39]:
# Query table - ALL
all_users = session.query(User).all()

# filter results - return one or none
# users = session.query(User).filter_by(id=3).one_or_none()
# print(users.name)

# filter results
#users = session.query(User).filter_by(id=3).all() = no logical operators allowed
# users = session.query(User).filter(User.id >= 3).all() # using logical operators
# users = session.query(User).filter(User.id >= 3, User.name == 'Radha').all() # using multiple conditions

#using where method - like SQL
# users = session.query(User).where(or_(User.age >= 17, User.name == 'Radha')).all() # with or condition
users = session.query(User).where(((User.age >= 17) | (User.name == 'Radha')) &  (User.id == 6)).all() # combination of and and or
print(len(all_users)) # print lenth of all results

for user in users:
    print(f"ID: {user.id} - Name: {user.name} - Age: {user.age}")

6
ID: 6 - Name: Radha - Age: 42


In [19]:
# update records
user = session.query(User).filter_by(id=2).one_or_none()
print(user.name)
user.age=45
session.commit()

Paneer


In [21]:
#delete record
session.delete(user)
session.commit()

In [25]:
# ordering results

# users = session.query(User).order_by(User.age).all() # assending
users = session.query(User).order_by(User.age.desc(), User.name).all() # dessending
for user in users:
    print(f"ID: {user.id} - Name: {user.name} - Age: {user.age}")


ID: 1 - Name: Paneer - Age: 44
ID: 3 - Name: Radha - Age: 42
ID: 4 - Name: Neha - Age: 17


In [55]:
# group by 
# users = session.query(User).all()
users = session.query(User.age, func.count(User.id)).group_by(User.age).all()
print(users)
print(len(users))

[(17, 2), (42, 2), (44, 2)]
3


In [59]:
# chaining
users = session.query(User.age).filter(User.age > 24).filter(User.age < 50).all()
print(users)

users_tuple = (
    session.query(User.age, func.count(User.id))
    .filter(User.age > 24)
    .order_by(User.age)
    .filter(User.age < 50)
    .group_by(User.age)
    .all()
)

for age, count in users_tuple:
    print(f"Age: {age} - {count} users")

[(44,), (42,), (44,), (42,)]
Age: 42 - 2 users
Age: 44 - 2 users


In [ ]:
# one to many relationship

# unmapped method

class BaseModel(Base):
    __abstract__ = True
    __allow_unmapped__ = True

    id = Column(Integer, primary_key=True)

class Address(BaseModel):
    __tablename__ = "adressess"
    city = Column(String)
    state = Column(String)
    zip_code = Column(Integer)
    user_id = Column(ForeignKey("nusers.id"))

    def __repr__(self):
        return f"<Address(id={self.id}, city='{self.city}')>"

class Nuser(BaseModel):
    __tablename__ = "nusers"
    name = Column(String)
    age = Column(Integer)
    adresses = relationship(Address)

    def __repr__(self):
        return f"<Nuser(id={self.id}, username='{self.name}')>"

Base.metadata.create_all(engine)

In [1]:
import pandas as pd
